In [1]:
from torchvision import models
import torch

import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt

resnet18_pretrained = models.resnet18(pretrained=True)

print(resnet18_pretrained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
num_classes= 18
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_classes)

In [3]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, transform, json_data):
        self.img_paths = img_paths
        self.transform = transform
        self.json_data = json_data
        
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        label = json_data[index//7]['img'][index%7]['class']
            
        return image, label

    def __len__(self):
        return len(self.img_paths)

In [4]:
import json
with open("train.json", "r") as f:
    json_data = json.load(f)
json_data = json_data['train']

In [ ]:
image_paths = []
for image in json_data:
    for img in image['img']:
        print(img['path'])
        image_paths.append(img['path'])

In [9]:
image_paths[:10]

['input/data/train/images/005231_male_Asian_19/incorrect_mask.jpg',
 'input/data/train/images/005231_male_Asian_19/mask1.jpg',
 'input/data/train/images/005231_male_Asian_19/mask2.jpg',
 'input/data/train/images/005231_male_Asian_19/mask3.jpg',
 'input/data/train/images/005231_male_Asian_19/mask4.jpg',
 'input/data/train/images/005231_male_Asian_19/mask5.jpg',
 'input/data/train/images/005231_male_Asian_19/normal.jpg',
 'input/data/train/images/004245_female_Asian_49/incorrect_mask.jpg',
 'input/data/train/images/004245_female_Asian_49/mask1.jpg',
 'input/data/train/images/004245_female_Asian_49/mask2.jpg']

In [5]:
# meta 데이터와 이미지 경로를 불러옵니다.
path_dir = 'input/data/train/images'

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = []
for image in json_data:
    for img in image['img']:
        image_paths.append(img['path'])
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TrainDataset(image_paths, transform,json_data)

loader = DataLoader(
    dataset,
    batch_size=100,
    shuffle=False
)

device = torch.device('cuda')
model = resnet18_pretrained.to(device)
model.train()

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 10 == 9:
            print('{},{:.5f} loss: {:.3f}'.format(epoch+1, i+1, running_loss/10))
            running_loss=0.0
    print('Finished')


1,10.00000 loss: 1.934
1,110.00000 loss: 9.732
Finished
2,10.00000 loss: 0.536
2,110.00000 loss: 6.367
Finished
3,10.00000 loss: 0.418
3,110.00000 loss: 5.308
Finished
4,10.00000 loss: 0.333
4,110.00000 loss: 4.281
Finished
5,10.00000 loss: 0.360
5,110.00000 loss: 3.897
Finished
6,10.00000 loss: 0.260
6,110.00000 loss: 3.350
Finished
7,10.00000 loss: 0.288
7,110.00000 loss: 3.349
Finished
8,10.00000 loss: 0.162
8,110.00000 loss: 2.586
Finished
9,10.00000 loss: 0.140
9,110.00000 loss: 2.221
Finished
10,10.00000 loss: 0.181
10,110.00000 loss: 2.069
Finished


In [6]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [7]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission3.csv'), index=False)
print('test inference is done!')

test inference is done!
